In [1]:
from pyomo.environ import *
import os
model = ConcreteModel()
solverexe = "gurobi"
infinity = float('inf')
# dirsolver = r"C:\Users\ch9fod\Documents\GitHub\ED\solvers"
# datafile = "data1.dat"

In [2]:
model.Bus = RangeSet(4, doc='Buses')
model.G = RangeSet(3, doc='Generators')
#parameters
model.a = Param(model.G, initialize={1:20.0 , 2:25.0 , 3:40.0},
                doc='Parameter a')
model.b = Param(model.G, initialize={1:0.05 , 2:0.10 , 3:0.20},
                doc='Parameter b')
model.Pmin = Param(model.G, initialize={1:20.0 , 2:20.0 , 3:30.0},
                   doc='Parameter Pmin')
model.Pmax = Param(model.G, initialize={1:400.0 , 2:300.0 , 3:250.0},
                   doc='Parameter Pmax')
model.Loads = Param(model.Bus, initialize={1:0.0 , 2:0.0 , 3:200.0 , 4:400.0},
                    doc='Loads for each Bus')
#lone parameter, constant
model.D = Param(initialize=600.0, doc='Load')
model.BaseP = Param(initialize=100.0, doc='Base Power')
model.PFmax = Param(initialize=300.0, doc='Max Power Flow')
model.PFmin = Param(initialize=-300.0, doc='Min Power Flow')
dtab = {
    (1,2) : 0.007,
    (2,3) : 0.01,
    (3,4) : 0.006,
    (4,1) : 0.05,
    (2,1) : 0.007,
    (3,2) : 0.01,
    (4,3) : 0.006,
    (1,4) : 0.05,
}
model.X = Param(model.Bus, model.Bus, initialize=dtab,
                doc='Reactance between buses', default=infinity)

In [3]:
# variables
model.P = Var(model.G, doc='Generation per Generator')
model.Angles = Var(model.Bus, doc='Angles of Buses')

In [4]:
#constraints
def generation_r(model,i):
    return (model.Pmin[i] , model.P[i] , model.Pmax[i])
model.GenCon = Constraint(model.G, rule = generation_r)
def demand_r(model):
    return model.D == sum(model.P[i] for i in model.G)
model.DemandCon = Constraint(model.G, rule = demand_r)
def slack_r(model):
    return model.Angles[1] == 0
model.SlackBusCon = Constraint(rule=slack_r)
def bus_r(model,i):
    if i == 4:
        return(-model.Loads[i] ==
               ((model.BaseP/model.X[i,i-1])*
                (model.Angles[i]-model.Angles[i-1]) +
                (model.BaseP/model.X[i,i-3])*
                (model.Angles[i]-model.Angles[i-3])))
    elif i == 1:
        return((model.P[i] - model.Loads[i]) ==
               ((model.BaseP/model.X[i,i+1])*
                (model.Angles[i]-model.Angles[i+1]) +
                (model.BaseP/model.X[i,i+3])*
                (model.Angles[i]-model.Angles[i+3])))
    else:
        return((model.P[i] - model.Loads[i]) ==
               ((model.BaseP/model.X[i,i+1])*
                (model.Angles[i]-model.Angles[i+1]) +
                (model.BaseP/model.X[i,i-1])*
                (model.Angles[i]-model.Angles[i-1])))
model.BusCon = Constraint(model.Bus, rule=bus_r)
def flow_r(model,i,j):
    if i == j:
        return Constraint.Skip
    return (model.PFmin ,
            (model.BaseP/model.X[i,j])*(model.Angles[i]--model.Angles[j]),
            model.PFmax)
model.FlowCon = Constraint(model.Bus, model.Bus, rule=flow_r)

In [5]:
#objective
def cost_rule(model):
    return sum(model.a[i]*model.P[i] +
               0.5*model.b[i]*model.P[i]**2
               for i in model.G)
#default is to minimize
model.OBJ = Objective(rule=cost_rule)

In [6]:
if solverexe == "gurobi":
    solver = SolverFactory(solverexe)
else:
    solver = SolverFactory(solverexe,
                           executable=os.path.join(dirsolver, solverexe))
instance = model.create_instance()
instance.dual = Suffix(direction=Suffix.IMPORT)
results = solver.solve(instance)

	constructed model; returning a clone of the current model instance.
WARNING - Loading a SolverResults object with a warning status into model=unknown; message from solver=Problem proven to be infeasible or unbounded.


In [7]:
instance.display()

Model unknown

  Variables:
    P : Generation per Generator
        Size=3, Index=G
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
    Angles : Angles of Buses
        Size=4, Index=Bus
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 :  None :  None :  None : False :  True :  Reals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
ERROR: evaluating expression: No value for uninitialized NumericValue object P[1]
	    (expression: 20.0*P[1] + 0.025*P[1]**2.0 + 25.0*P[2] + 0.05*P[2]**2.0 + 40.0*P[3] + 0.1*P[3]**2.0)
ERROR: evaluating object as numeric value: OBJ
	    (object: <class 

TypeError: not all arguments converted during string formatting